# VacationPy
This notebook uses the weather data colllected in WeatherPy notebook to find an ideal vacation hotel based on some wather conditions. The notebook uses the gamps Python module as well as the google places API. 
***
## Set Up
First we load dependencies and the weather_data dataframe which was created in the WeatherPy notebook.

In [1]:
# Import dependencies, api_key and configure gmaps with api_key
import gmaps
import pandas as pd
from api_keys import gkey
import random as r
import numpy as np
import requests
gmaps.configure(api_key=gkey)

In [2]:
# Load the DataFrame from WeatherPy
weather_data = pd.read_csv('../WeatherPy/Output/weather_data.csv')
weather_data.head()

,city,latitude (°N),longitude (°E),temperature (°F),humidity (%),cloudiness (%),wind speed (mph)
0,"Hambantota, LK",6.124913,81.124256,77.32,100,20,5.75
1,"Bagula, IN",23.333869,88.642096,84.90,74,100,3.18
2,"Airai, PW",7.359934,134.560115,79.20,83,75,4.61
3,"Tual, ID",-5.644309,132.741682,79.34,82,100,21.56
4,"Isla Vista, US",34.411513,-119.856213,80.08,59,74,4.18


We also will need the get_closet_hotel function to help call the google places API.

In [3]:
# Create a function to search location for a hotel using google places api
def get_closest_hotel(lat, lon):

    # Base URL for places api + parameters
    places_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={gkey}&radius=5000&type=lodging&location={lat},{lon}'
    
    # Call api and access the first result '[0]'
    first_response = requests.get(places_url).json()['results'][0]
    
    # Package and return the results as a dictionary 
    return {'hotel': first_response['name'], 'latitude': first_response['geometry']['location']['lat'], 'longitude': first_response['geometry']['location']['lng']}

## Humidity Heat Map
After getting set up we look at humidity data collected for 500 random cities in a heatmap. Generally, it looks like cities closer to the ocean have higher humidity which is reasonable.

In [4]:
# Build heat map layer with info from df
heatmap = gmaps.heatmap_layer(weather_data[['latitude (°N)', 'longitude (°E)']], weights=weather_data['humidity (%)'], max_intensity=100, point_radius=10)
ideal_vacation_map = gmaps.figure()
ideal_vacation_map.add_layer(heatmap)

# Show heat map
ideal_vacation_map

Figure(layout=FigureLayout(height='420px'))

## Hotel Markers
Next we look "ideal" weather conditions: Temperature between 70 and 80°F, wind speed less than 5 mph, and 0% cloudiness.

In [5]:
# Narrow down weather to ideal conditions
ideal_weather = weather_data.loc[(np.abs(weather_data['temperature (°F)']-75)<=5)&(weather_data['wind speed (mph)']<5)&(weather_data['cloudiness (%)']==0),:].reset_index(drop=True)
ideal_weather

,city,latitude (°N),longitude (°E),temperature (°F),humidity (%),cloudiness (%),wind speed (mph)
0,"Limassol, CY",34.685290,33.033266,71.55,73,0,4.61
1,"Aybak, AF",36.273067,68.021845,78.40,20,0,4.81
2,"Kashan, IR",33.987653,51.442886,78.42,16,0,0.00
3,"Antalya, TR",36.900964,30.695485,76.44,69,0,3.44
4,"Minjivan, AZ",39.028600,46.715779,76.39,46,0,0.58
5,"Woodward, US",41.856930,-93.921896,70.27,78,0,3.44
6,"Khudat, AZ",41.632895,48.680353,77.90,43,0,3.24
7,"Yining City, CN",43.905203,81.274780,74.55,22,0,4.63
8,"Trinidad, CU",21.805205,-79.984580,79.59,78,0,2.33


Using a for loop and the get_closest_hotel method, we obtain a marker layer for the narrowe down dataset. We plot that on top of the heatmap layer to see where the most favorable hotel locations are at. 

In [7]:
# Intialize 2 lists
locations = []
hotel_info_list = []
for lat, lon, city in zip(ideal_weather['latitude (°N)'], ideal_weather['longitude (°E)'], ideal_weather['city']):
    try:
        hotel = get_closest_hotel(lat, lon)
    except:
        hotel = {'hotel': 'No Hotel Found!', 'latitude': lat, 'longitude': lon}
    locations.append((hotel['latitude'], hotel['longitude']))
    hotel_info = f"Hotel: {hotel['hotel']} City: {city.split()[0]} Country: {city.split()[1]}"
    hotel_info_list.append(hotel_info)

# Create the markers and show the figure
hotel_markers = gmaps.marker_layer(locations=locations, info_box_content=hotel_info_list)
ideal_vacation_map.add_layer(hotel_markers)
ideal_vacation_map

Figure(layout=FigureLayout(height='420px'))